In [2]:
import aiohttp
import asyncio
import pandas as pd
from datetime import datetime

URL = "https://wsc.virginmobile.ae/numbers/search"

HEADERS = {
    "Accept": "application/json",
    "Channel-Id": "Web",
    "ChannelID": "Web",
    "Content-Type": "application/json",
    "Origin": "https://www.virginmobile.ae",
    "Referer": "https://www.virginmobile.ae/",
    "Session-Id": "6112a785-ab05-488f-baca-01983e3cc2cf",
    "User-Agent": "Mozilla/5.0",
    "Virgin-Device-Id": "730899726",
    "Virgin-Device-Os": "Web",
    "Virgin-Lang": "en",
    "accept-version": "v34"
}

# Generate sample list (you can expand to 1M later)
patterns = [f"0585{i:06d}" for i in range(1000)]  

results = []
start_time = datetime.now()

async def fetch(session, pattern):
    payload = {"per_page": 1, "page": 1, "pattern": pattern, "vanity_level_ids": "3"}
    try:
        async with session.post(URL, headers=HEADERS, json=payload, timeout=10) as resp:
            if resp.status in (200, 201):
                data = await resp.json()
                # Virgin returns numbers in "data" list
                valid = "valid" if data.get("data") else "not valid"
            else:
                valid = f"error ({resp.status})"
    except Exception as e:
        valid = f"error ({str(e)})"

    return {"pattern": pattern, "status": valid}

async def run():
    connector = aiohttp.TCPConnector(limit=100)  # limit concurrent connections
    async with aiohttp.ClientSession(connector=connector) as session:
        tasks = [fetch(session, p) for p in patterns]
        for i, task in enumerate(asyncio.as_completed(tasks), 1):
            res = await task
            results.append(res)
            if i % 100 == 0:
                print(f"[{i}/{len(patterns)}] processed")

asyncio.run(run())

# Save results
df = pd.DataFrame(results)
df.to_excel("virgin_numbers_async.xlsx", index=False)

# Summary
valid_count = df[df['status'] == "valid"].shape[0]
invalid_count = df[df['status'] == "not valid"].shape[0]
error_count = df[df['status'].str.contains("error")].shape[0]

print("\n--- Scan Complete ---")
print(f"✅ Valid: {valid_count}")
print(f"❌ Not Valid: {invalid_count}")
print(f"⚠️ Errors: {error_count}")
print(f"🕒 Total Time: {datetime.now() - start_time}")


RuntimeError: asyncio.run() cannot be called from a running event loop

In [4]:
!pip install aiohttp nest_asyncio openpyxl

In [6]:
!pip install aiohttp nest_asyncio openpyxl

import aiohttp
import asyncio
import pandas as pd
import nest_asyncio
from datetime import datetime

# Allow nested event loops (needed for Jupyter/Colab)
nest_asyncio.apply()

# Virgin Mobile number search API
URL = "https://wsc.virginmobile.ae/numbers/search"

HEADERS = {
    "Accept": "application/json",
    "Channel-Id": "Web",
    "ChannelID": "Web",
    "Content-Type": "application/json",
    "Origin": "https://www.virginmobile.ae",
    "Referer": "https://www.virginmobile.ae/",
    "Session-Id": "6112a785-ab05-488f-baca-01983e3cc2cf",  # rotate if blocked
    "User-Agent": "Mozilla/5.0",
    "Virgin-Device-Id": "730899726",
    "Virgin-Device-Os": "Web",
    "Virgin-Lang": "en",
    "accept-version": "v34"
}

# ⚡ CHANGE THIS: total range to scan (here just 1000 for testing)
# Full range would be: range(1000000)
patterns = [f"0585{i:06d}" for i in range(1000)]

results = []
start_time = datetime.now()

async def fetch(session, pattern):
    payload = {"per_page": 1, "page": 1, "pattern": pattern, "vanity_level_ids": "3"}
    try:
        async with session.post(URL, headers=HEADERS, json=payload, timeout=10) as resp:
            if resp.status in (200, 201):
                data = await resp.json()
                # API usually returns numbers inside "data"
                valid = "valid" if data.get("data") else "not valid"
            else:
                valid = f"error ({resp.status})"
    except Exception as e:
        valid = f"error ({str(e)})"
    return {"pattern": pattern, "status": valid}

async def process_batch(batch_patterns, batch_index):
    batch_results = []
    connector = aiohttp.TCPConnector(limit=100)  # concurrency limit
    async with aiohttp.ClientSession(connector=connector) as session:
        tasks = [fetch(session, p) for p in batch_patterns]
        for i, task in enumerate(asyncio.as_completed(tasks), 1):
            res = await task
            batch_results.append(res)
            if i % 100 == 0:
                print(f"Batch {batch_index}: [{i}/{len(batch_patterns)}] processed")

    # Save after each batch
    df = pd.DataFrame(batch_results)
    df.to_excel(f"virgin_numbers_batch_{batch_index}.xlsx", index=False)
    print(f"✅ Saved batch {batch_index} ({len(batch_results)} records)")
    return batch_results

async def main():
    batch_size = 10000   # adjust as needed
    all_results = []
    for i in range(0, len(patterns), batch_size):
        batch_patterns = patterns[i:i+batch_size]
        batch_index = (i // batch_size) + 1
        print(f"\n🚀 Starting batch {batch_index} with {len(batch_patterns)} numbers...")
        batch_results = await process_batch(batch_patterns, batch_index)
        all_results.extend(batch_results)

    return all_results

# Run in Jupyter/Colab
all_results = await main()

# Save full result
df = pd.DataFrame(all_results)
df.to_excel("virgin_numbers_all.xlsx", index=False)

# Summary
valid_count = df[df['status'] == "valid"].shape[0]
invalid_count = df[df['status'] == "not valid"].shape[0]
error_count = df[df['status'].str.contains("error")].shape[0]

print("\n--- Scan Complete ---")
print(f"✅ Valid: {valid_count}")
print(f"❌ Not Valid: {invalid_count}")
print(f"⚠️ Errors: {error_count}")
print(f"🕒 Total Time: {datetime.now() - start_time}")



🚀 Starting batch 1 with 1000 numbers...
Batch 1: [100/1000] processed
Batch 1: [200/1000] processed
Batch 1: [300/1000] processed
Batch 1: [400/1000] processed
Batch 1: [500/1000] processed
Batch 1: [600/1000] processed
Batch 1: [700/1000] processed
Batch 1: [800/1000] processed
Batch 1: [900/1000] processed
Batch 1: [1000/1000] processed
✅ Saved batch 1 (1000 records)

--- Scan Complete ---
✅ Valid: 0
❌ Not Valid: 1000
⚠️ Errors: 0
🕒 Total Time: 0:00:05.534034


In [16]:
import requests
import pandas as pd
import time
import random
from datetime import datetime
import os

# Virgin Mobile number search API
URL = "https://wsc.virginmobile.ae/numbers/search"

HEADERS = {
    "Accept": "application/json",
    "Channel-Id": "Web",
    "ChannelID": "Web",
    "Content-Type": "application/json",
    "Origin": "https://www.virginmobile.ae",
    "Referer": "https://www.virginmobile.ae/",
    "Session-Id": "6112a785-ab05-488f-baca-01983e3cc2cf",  # Update if needed
    "User-Agent": "Mozilla/5.0",
    "Virgin-Device-Id": "730899726",
    "Virgin-Device-Os": "Web",
    "Virgin-Lang": "en",
    "accept-version": "v34"
}

patterns = [f"0585{i:06d}" for i in range(1000000)]

# File for valid numbers
valid_file = "virgin_valid_numbers.xlsx"

# Check if file already exists to append to it
if os.path.exists(valid_file):
    valid_df = pd.read_excel(valid_file)
else:
    valid_df = pd.DataFrame(columns=["pattern", "timestamp", "exact_valid_number"])

start_time = datetime.now()

for idx, pattern in enumerate(patterns, 1):
    try:
        payload = {
            "per_page": 1,
            "page": 1,
            "pattern": pattern,
            "vanity_level_ids": "3"
        }

        resp = requests.post(URL, headers=HEADERS, json=payload)

        if resp.status_code in (200, 201):
            data = resp.json()
            is_valid = data.get("is_recommended_number", False)

            if is_valid:
                # Extract MSISDN numbers if present
                numbers = data.get("numbers", [])
                msisdns = [n.get("MSISDN") for n in numbers if "MSISDN" in n]

                # Join if multiple numbers exist
                msisdns_str = ", ".join(msisdns) if msisdns else ""

                new_row = pd.DataFrame([{
                    "pattern": pattern,
                    "timestamp": datetime.now(),
                    "exact_valid_number": msisdns_str
                }])

                valid_df = pd.concat([valid_df, new_row], ignore_index=True)
                valid_df.to_excel(valid_file, index=False)

                status = f"valid → saved {msisdns_str}"
            else:
                status = "not valid"
        else:
            status = f"error ({resp.status_code})"

    except Exception as e:
        status = f"error ({str(e)})"

    # Display progress
    print(f"[{idx}/{len(patterns)}] {pattern}: {status}")

    # Optional: Delay to avoid hammering the API
    # time.sleep(random.uniform(0.1, 0.3))

print("\n--- Scan Complete ---")
print(f"🕒 Total Time: {datetime.now() - start_time}")


[1/1000000] 0585000000: not valid
[2/1000000] 0585000001: not valid
[3/1000000] 0585000002: not valid
[4/1000000] 0585000003: not valid
[5/1000000] 0585000004: not valid
[6/1000000] 0585000005: not valid
[7/1000000] 0585000006: not valid
[8/1000000] 0585000007: not valid
[9/1000000] 0585000008: not valid
[10/1000000] 0585000009: not valid
[11/1000000] 0585000010: not valid
[12/1000000] 0585000011: not valid
[13/1000000] 0585000012: not valid
[14/1000000] 0585000013: not valid
[15/1000000] 0585000014: not valid
[16/1000000] 0585000015: not valid
[17/1000000] 0585000016: not valid
[18/1000000] 0585000017: not valid
[19/1000000] 0585000018: not valid
[20/1000000] 0585000019: not valid
[21/1000000] 0585000020: not valid
[22/1000000] 0585000021: not valid
[23/1000000] 0585000022: not valid
[24/1000000] 0585000023: not valid
[25/1000000] 0585000024: not valid
[26/1000000] 0585000025: not valid
[27/1000000] 0585000026: not valid
[28/1000000] 0585000027: not valid
[29/1000000] 0585000028: vali

KeyboardInterrupt: 

In [18]:
import requests
import pandas as pd
from datetime import datetime
import os

# Virgin Mobile number search API
URL = "https://wsc.virginmobile.ae/numbers/search"

HEADERS = {
    "Accept": "application/json",
    "Channel-Id": "Web",
    "ChannelID": "Web",
    "Content-Type": "application/json",
    "Origin": "https://www.virginmobile.ae",
    "Referer": "https://www.virginmobile.ae/",
    "Session-Id": "6112a785-ab05-488f-baca-01983e3cc2cf",  # Update if needed
    "User-Agent": "Mozilla/5.0",
    "Virgin-Device-Id": "730899726",
    "Virgin-Device-Os": "Web",
    "Virgin-Lang": "en",
    "accept-version": "v34"
}

# Output file
valid_file = "virgin_valid_numbers.xlsx"

# Load existing numbers (to avoid duplicates)
if os.path.exists(valid_file):
    valid_df = pd.read_excel(valid_file, dtype=str)
else:
    valid_df = pd.DataFrame(columns=["timestamp", "exact_valid_number"])

# Keep a Python set of already saved numbers for quick lookup
existing_numbers = set(valid_df["exact_valid_number"].dropna().tolist())

print(f"📊 Already have {len(existing_numbers)} numbers in file.")

start_time = datetime.now()

# Example strategy: loop pages instead of brute-forcing 1M patterns
# Adjust `pattern` to search different prefixes
pattern = "5"  
vanity_level = "4"
per_page = 335

for page in range(1, ):  # adjust max pages as needed
    try:
        payload = {
            "per_page": per_page,
            "page": page,
            "pattern": pattern,
            "vanity_level_ids": vanity_level
        }

        resp = requests.post(URL, headers=HEADERS, json=payload)

        if resp.status_code not in (200, 201):
            print(f"❌ Error {resp.status_code} on page {page}")
            continue

        data = resp.json()
        numbers = data.get("numbers", [])

        if not numbers:
            print(f"⚠️ No numbers on page {page}, stopping.")
            break

        new_rows = []
        for n in numbers:
            msisdn = n.get("MSISDN")
            if not msisdn or msisdn in existing_numbers:
                continue  # skip duplicates or empty
            existing_numbers.add(msisdn)

            new_rows.append({
                "timestamp": datetime.now(),
                "exact_valid_number": msisdn
            })

        if new_rows:
            valid_df = pd.concat([valid_df, pd.DataFrame(new_rows)], ignore_index=True)
            valid_df.to_excel(valid_file, index=False)
            print(f"✅ Page {page}: saved {len(new_rows)} new numbers")

        else:
            print(f"ℹ️ Page {page}: no new numbers")

    except Exception as e:
        print(f"⚠️ Error on page {page}: {e}")
        continue

print("\n--- Scan Complete ---")
print(f"🕒 Total Time: {datetime.now() - start_time}")
print(f"📊 Final unique numbers saved: {len(existing_numbers)}")


📊 Already have 0 numbers in file.
❌ Error 400 on page 1
❌ Error 400 on page 2
❌ Error 400 on page 3
❌ Error 400 on page 4
❌ Error 400 on page 5


/var/folders/q0/s8crqpxd1hdb8d_2t98dwk9m0000gn/T/ipykernel_21703/1903586092.py:81: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  valid_df = pd.concat([valid_df, pd.DataFrame(new_rows)], ignore_index=True)


✅ Page 6: saved 199 new numbers
❌ Error 400 on page 7
❌ Error 400 on page 8
❌ Error 400 on page 9
❌ Error 400 on page 10
❌ Error 400 on page 11
❌ Error 400 on page 12
❌ Error 400 on page 13
❌ Error 400 on page 14
❌ Error 400 on page 15
❌ Error 400 on page 16
❌ Error 400 on page 17
❌ Error 400 on page 18
❌ Error 400 on page 19
❌ Error 400 on page 20
❌ Error 400 on page 21
❌ Error 400 on page 22
❌ Error 400 on page 23
❌ Error 400 on page 24
❌ Error 400 on page 25
❌ Error 400 on page 26
❌ Error 400 on page 27
❌ Error 400 on page 28
❌ Error 400 on page 29
❌ Error 400 on page 30
❌ Error 400 on page 31
❌ Error 400 on page 32
❌ Error 400 on page 33
❌ Error 400 on page 34
❌ Error 400 on page 35
❌ Error 400 on page 36
❌ Error 400 on page 37
❌ Error 400 on page 38
❌ Error 400 on page 39
❌ Error 400 on page 40
❌ Error 400 on page 41
❌ Error 400 on page 42
❌ Error 400 on page 43
❌ Error 400 on page 44
❌ Error 400 on page 45
❌ Error 400 on page 46
❌ Error 400 on page 47
❌ Error 400 on page 48
❌ Err

In [22]:
x = json.dump({"numbers":[{"MSISDN":"0585111108","VanityLevel":4},{"MSISDN":"0585111102","VanityLevel":4},{"MSISDN":"0585111105","VanityLevel":4},{"MSISDN":"0585111106","VanityLevel":4},{"MSISDN":"0585111107","VanityLevel":4},{"MSISDN":"0585111109","VanityLevel":4},{"MSISDN":"0585111124","VanityLevel":4},{"MSISDN":"0585111125","VanityLevel":4},{"MSISDN":"0585111132","VanityLevel":4},{"MSISDN":"0585111134","VanityLevel":4},{"MSISDN":"0585111135","VanityLevel":4},{"MSISDN":"0585111136","VanityLevel":4},{"MSISDN":"0585111139","VanityLevel":4},{"MSISDN":"0585111141","VanityLevel":4},{"MSISDN":"0585111143","VanityLevel":4},{"MSISDN":"0585111145","VanityLevel":4},{"MSISDN":"0585111146","VanityLevel":4},{"MSISDN":"0585111147","VanityLevel":4},{"MSISDN":"0585111148","VanityLevel":4},{"MSISDN":"0585111149","VanityLevel":4},{"MSISDN":"0585111153","VanityLevel":4},{"MSISDN":"0585111154","VanityLevel":4},{"MSISDN":"0585111156","VanityLevel":4},{"MSISDN":"0585111157","VanityLevel":4},{"MSISDN":"0585111159","VanityLevel":4},{"MSISDN":"0585111160","VanityLevel":4},{"MSISDN":"0585111162","VanityLevel":4},{"MSISDN":"0585111163","VanityLevel":4},{"MSISDN":"0585111164","VanityLevel":4},{"MSISDN":"0585111165","VanityLevel":4},{"MSISDN":"0585111167","VanityLevel":4},{"MSISDN":"0585111168","VanityLevel":4},{"MSISDN":"0585111169","VanityLevel":4},{"MSISDN":"0585111170","VanityLevel":4},{"MSISDN":"0585111173","VanityLevel":4},{"MSISDN":"0585111174","VanityLevel":4},{"MSISDN":"0585111176","VanityLevel":4},{"MSISDN":"0585111178","VanityLevel":4},{"MSISDN":"0585111179","VanityLevel":4},{"MSISDN":"0585111182","VanityLevel":4},{"MSISDN":"0585111183","VanityLevel":4},{"MSISDN":"0585111184","VanityLevel":4},{"MSISDN":"0585111186","VanityLevel":4},{"MSISDN":"0585111187","VanityLevel":4},{"MSISDN":"0585111189","VanityLevel":4},{"MSISDN":"0585111192","VanityLevel":4},{"MSISDN":"0585111195","VanityLevel":4},{"MSISDN":"0585111197","VanityLevel":4},{"MSISDN":"0585111198","VanityLevel":4},{"MSISDN":"0585122220","VanityLevel":4},{"MSISDN":"0585122223","VanityLevel":4},{"MSISDN":"0585122224","VanityLevel":4},{"MSISDN":"0585122226","VanityLevel":4},{"MSISDN":"0585122227","VanityLevel":4},{"MSISDN":"0585122228","VanityLevel":4},{"MSISDN":"0585122229","VanityLevel":4},{"MSISDN":"0585133330","VanityLevel":4},{"MSISDN":"0585133331","VanityLevel":4},{"MSISDN":"0585133332","VanityLevel":4},{"MSISDN":"0585133334","VanityLevel":4},{"MSISDN":"0585133336","VanityLevel":4},{"MSISDN":"0585133337","VanityLevel":4},{"MSISDN":"0585144443","VanityLevel":4},{"MSISDN":"0585144446","VanityLevel":4},{"MSISDN":"0585144447","VanityLevel":4},{"MSISDN":"0585144448","VanityLevel":4},{"MSISDN":"0585144449","VanityLevel":4},{"MSISDN":"0585166661","VanityLevel":4},{"MSISDN":"0585166663","VanityLevel":4},{"MSISDN":"0585166664","VanityLevel":4},{"MSISDN":"0585166665","VanityLevel":4},{"MSISDN":"0585166667","VanityLevel":4},{"MSISDN":"0585166668","VanityLevel":4},{"MSISDN":"0585177772","VanityLevel":4},{"MSISDN":"0585177774","VanityLevel":4},{"MSISDN":"0585177776","VanityLevel":4},{"MSISDN":"0585188883","VanityLevel":4},{"MSISDN":"0585188884","VanityLevel":4},{"MSISDN":"0585193333","VanityLevel":4},{"MSISDN":"0585194444","VanityLevel":4},{"MSISDN":"0585199993","VanityLevel":4},{"MSISDN":"0585199994","VanityLevel":4},{"MSISDN":"0585199997","VanityLevel":4},{"MSISDN":"0585199998","VanityLevel":4},{"MSISDN":"0585222251","VanityLevel":4},{"MSISDN":"0585333351","VanityLevel":4},{"MSISDN":"0585511114","VanityLevel":4},{"MSISDN":"0585511116","VanityLevel":4},{"MSISDN":"0585511118","VanityLevel":4},{"MSISDN":"0585511119","VanityLevel":4},{"MSISDN":"0585555107","VanityLevel":4},{"MSISDN":"0585555113","VanityLevel":4},{"MSISDN":"0585555120","VanityLevel":4},{"MSISDN":"0585555121","VanityLevel":4},{"MSISDN":"0585555126","VanityLevel":4},{"MSISDN":"0585555133","VanityLevel":4},{"MSISDN":"0585555139","VanityLevel":4},{"MSISDN":"0585555140","VanityLevel":4},{"MSISDN":"0585555141","VanityLevel":4},{"MSISDN":"0585555143","VanityLevel":4},{"MSISDN":"0585555153","VanityLevel":4},{"MSISDN":"0585555154","VanityLevel":4},{"MSISDN":"0585555156","VanityLevel":4},{"MSISDN":"0585555168","VanityLevel":4},{"MSISDN":"0585555169","VanityLevel":4},{"MSISDN":"0585555170","VanityLevel":4},{"MSISDN":"0585555171","VanityLevel":4},{"MSISDN":"0585555173","VanityLevel":4},{"MSISDN":"0585555178","VanityLevel":4},{"MSISDN":"0585555180","VanityLevel":4},{"MSISDN":"0585555185","VanityLevel":4},{"MSISDN":"0585555189","VanityLevel":4},{"MSISDN":"0585555191","VanityLevel":4},{"MSISDN":"0585555193","VanityLevel":4},{"MSISDN":"0585555195","VanityLevel":4},{"MSISDN":"0585555196","VanityLevel":4},{"MSISDN":"0585555651","VanityLevel":4},{"MSISDN":"0585555851","VanityLevel":4},{"MSISDN":"0585655551","VanityLevel":4},{"MSISDN":"0585666651","VanityLevel":4},{"MSISDN":"0585755551","VanityLevel":4},{"MSISDN":"0585955551","VanityLevel":4},{"MSISDN":"0585111103","VanityLevel":4},{"MSISDN":"0585111126","VanityLevel":4},{"MSISDN":"0585111128","VanityLevel":4},{"MSISDN":"0585111137","VanityLevel":4},{"MSISDN":"0585111142","VanityLevel":4},{"MSISDN":"0585111172","VanityLevel":4},{"MSISDN":"0585111190","VanityLevel":4},{"MSISDN":"0585111193","VanityLevel":4},{"MSISDN":"0585111194","VanityLevel":4},{"MSISDN":"0585122225","VanityLevel":4},{"MSISDN":"0585133335","VanityLevel":4},{"MSISDN":"0585133338","VanityLevel":4},{"MSISDN":"0585133339","VanityLevel":4},{"MSISDN":"0585155553","VanityLevel":4},{"MSISDN":"0585163333","VanityLevel":4},{"MSISDN":"0585166669","VanityLevel":4},{"MSISDN":"0585255551","VanityLevel":4},{"MSISDN":"0585355551","VanityLevel":4},{"MSISDN":"0585511112","VanityLevel":4},{"MSISDN":"0585511113","VanityLevel":4},{"MSISDN":"0585511117","VanityLevel":4},{"MSISDN":"0585551117","VanityLevel":4},{"MSISDN":"0585555102","VanityLevel":4},{"MSISDN":"0585555103","VanityLevel":4},{"MSISDN":"0585555104","VanityLevel":4},{"MSISDN":"0585555106","VanityLevel":4},{"MSISDN":"0585555109","VanityLevel":4},{"MSISDN":"0585555112","VanityLevel":4},{"MSISDN":"0585555114","VanityLevel":4},{"MSISDN":"0585555116","VanityLevel":4},{"MSISDN":"0585555117","VanityLevel":4},{"MSISDN":"0585555119","VanityLevel":4},{"MSISDN":"0585555122","VanityLevel":4},{"MSISDN":"0585555124","VanityLevel":4},{"MSISDN":"0585555127","VanityLevel":4},{"MSISDN":"0585555128","VanityLevel":4},{"MSISDN":"0585555129","VanityLevel":4},{"MSISDN":"0585555130","VanityLevel":4},{"MSISDN":"0585555131","VanityLevel":4},{"MSISDN":"0585555132","VanityLevel":4},{"MSISDN":"0585555134","VanityLevel":4},{"MSISDN":"0585555135","VanityLevel":4},{"MSISDN":"0585555136","VanityLevel":4},{"MSISDN":"0585555137","VanityLevel":4},{"MSISDN":"0585555142","VanityLevel":4},{"MSISDN":"0585555144","VanityLevel":4},{"MSISDN":"0585555145","VanityLevel":4},{"MSISDN":"0585555146","VanityLevel":4},{"MSISDN":"0585555147","VanityLevel":4},{"MSISDN":"0585555148","VanityLevel":4},{"MSISDN":"0585555149","VanityLevel":4},{"MSISDN":"0585555157","VanityLevel":4},{"MSISDN":"0585555160","VanityLevel":4},{"MSISDN":"0585555162","VanityLevel":4},{"MSISDN":"0585555163","VanityLevel":4},{"MSISDN":"0585555164","VanityLevel":4},{"MSISDN":"0585555165","VanityLevel":4},{"MSISDN":"0585555167","VanityLevel":4},{"MSISDN":"0585555172","VanityLevel":4},{"MSISDN":"0585555174","VanityLevel":4},{"MSISDN":"0585555175","VanityLevel":4},{"MSISDN":"0585555176","VanityLevel":4},{"MSISDN":"0585555179","VanityLevel":4},{"MSISDN":"0585555181","VanityLevel":4},{"MSISDN":"0585555182","VanityLevel":4},{"MSISDN":"0585555183","VanityLevel":4},{"MSISDN":"0585555184","VanityLevel":4},{"MSISDN":"0585555186","VanityLevel":4},{"MSISDN":"0585555187","VanityLevel":4},{"MSISDN":"0585555190","VanityLevel":4},{"MSISDN":"0585555192","VanityLevel":4},{"MSISDN":"0585555194","VanityLevel":4},{"MSISDN":"0585555197","VanityLevel":4},{"MSISDN":"0585555198","VanityLevel":4},{"MSISDN":"0585555199","VanityLevel":4},{"MSISDN":"0585555251","VanityLevel":4},{"MSISDN":"0585555351","VanityLevel":4},{"MSISDN":"0585555451","VanityLevel":4},{"MSISDN":"0585555951","VanityLevel":4},{"MSISDN":"0585777751","VanityLevel":4},{"MSISDN":"0585999951","VanityLevel":4}],"is_recommended_number":false})

NameError: name 'false' is not defined

In [32]:
import requests
import pandas as pd
import time
import random
from datetime import datetime
import os
import json

# Virgin Mobile number search API
URL = "https://wsc.virginmobile.ae/numbers/search"

HEADERS = {
    "Accept": "application/json",
    "Channel-Id": "Web",
    "ChannelID": "Web",
    "Content-Type": "application/json",
    "Origin": "https://www.virginmobile.ae",
    "Referer": "https://www.virginmobile.ae/",
    "Session-Id": "6112a785-ab05-488f-baca-01983e3cc2cf",  # Update if needed
    "User-Agent": "Mozilla/5.0",
    "Virgin-Device-Id": "730899726",
    "Virgin-Device-Os": "Web",
    "Virgin-Lang": "en",
    "accept-version": "v34"
}

# Vanity level to fetch
VANITY_LEVEL = 4
PER_PAGE = 50  # can increase to 1000 if API supports
SAVE_JSON_FILE = f"vanity_level_{VANITY_LEVEL}_pages.json"

start_time = datetime.now()
all_pages = {}
page = 1

while True:
    try:
        payload = {
            "per_page": PER_PAGE,
            "page": page,
            "pattern": "",
            "vanity_level_ids": str(VANITY_LEVEL)
        }
        print(payload)
        resp = requests.post(URL, headers=HEADERS, json=payload)
        resp.raise_for_status()
        data = resp.json().get("data", [])

        if not data:
            print(f"No more data on page {page}. Ending fetch.")
            break

        # Save this page separately
        all_pages[f"page_{page}"] = data
        print(f"[Page {page}] Fetched {len(data)} numbers")

        page += 1
        # time.sleep(random.uniform(0.2, 0.5))  # avoid hammering API

    except Exception as e:
        print(f"Error on page {page}: {e}")
        time.sleep(1)

# Save all pages to JSON
with open(SAVE_JSON_FILE, "w") as f:
    json.dump(all_pages, f, indent=4)

print(f"\n--- Fetch Complete ---")
print(f"Total pages fetched: {len(all_pages)}")
print(f"Saved to: {SAVE_JSON_FILE}")
print(f"🕒 Total Time: {datetime.now() - start_time}")


{'per_page': 50, 'page': 1, 'pattern': '', 'vanity_level_ids': '4'}
No more data on page 1. Ending fetch.

--- Fetch Complete ---
Total pages fetched: 0
Saved to: vanity_level_4_pages.json
🕒 Total Time: 0:00:00.281890


In [40]:
import requests
import pandas as pd
import time
import random
from datetime import datetime

# Virgin Mobile number search API
URL = "https://wsc.virginmobile.ae/numbers/search"

HEADERS = {
    "Accept": "application/json",
    "Channel-Id": "Web",
    "ChannelID": "Web",
    "Content-Type": "application/json",
    "Origin": "https://www.virginmobile.ae",
    "Referer": "https://www.virginmobile.ae/",
    "Session-Id": "6112a785-ab05-488f-baca-01983e3cc2cf",  # Update if needed
    "User-Agent": "Mozilla/5.0",
    "Virgin-Device-Id": "730899726",
    "Virgin-Device-Os": "Web",
    "Virgin-Lang": "en",
    "accept-version": "v34"
}

VANITY_LEVEL = 4
PER_PAGE = 50  # can increase to 1000 if API allows
EXCEL_FILE = f"vanity_level_{VANITY_LEVEL}_numbers.xlsx"

all_numbers = []
page = 1
start_time = datetime.now()

while True:
    try:
        payload = {
            "per_page": PER_PAGE,
            "page": page,
            "pattern": "",
            "vanity_level_ids": str(VANITY_LEVEL)
        }

        resp = requests.post(URL, headers=HEADERS, json=payload)
        print(resp.json())
        resp.raise_for_status()
        data = resp.json().get("numbers", [])

        if not data:
            print(f"No more data on page {page}. Ending fetch.")
            break

        # Extract MSISDN numbers and store with full info
        for item in data:
            msisdn = item.get("MSISDN")
            if msisdn:
                all_numbers.append({
                    "MSISDN": msisdn,
                    "vanity_level": VANITY_LEVEL,
                    "raw_data": item
                })

        print(f"[Page {page}] Fetched {len(data)} numbers. Total collected: {len(all_numbers)}")
        page += 1
        # time.sleep(random.uniform(0.2, 0.5))  # avoid hammering API

    except Exception as e:
        print(f"Error on page {page}: {e}")
        time.sleep(1)

# Convert to DataFrame and remove duplicates if any
df = pd.DataFrame(all_numbers).drop_duplicates(subset="MSISDN")

# Save to Excel
df.to_excel(EXCEL_FILE, index=False)

print(f"\n--- Fetch Complete ---")
print(f"Total numbers fetched: {len(df)}")
print(f"Saved to Excel: {EXCEL_FILE}")
print(f"🕒 Total Time: {datetime.now() - start_time}")


{'numbers': [{'MSISDN': '0585488889', 'VanityLevel': 4}, {'MSISDN': '0585613333', 'VanityLevel': 4}, {'MSISDN': '0585999947', 'VanityLevel': 4}, {'MSISDN': '0585482222', 'VanityLevel': 4}, {'MSISDN': '0585577776', 'VanityLevel': 4}, {'MSISDN': '0585966663', 'VanityLevel': 4}, {'MSISDN': '0585633334', 'VanityLevel': 4}, {'MSISDN': '0585233330', 'VanityLevel': 4}, {'MSISDN': '0585555318', 'VanityLevel': 4}, {'MSISDN': '0585333394', 'VanityLevel': 4}, {'MSISDN': '0585066669', 'VanityLevel': 4}, {'MSISDN': '0585623333', 'VanityLevel': 4}, {'MSISDN': '0585555370', 'VanityLevel': 4}, {'MSISDN': '0585555669', 'VanityLevel': 4}, {'MSISDN': '0585555985', 'VanityLevel': 4}, {'MSISDN': '0585377774', 'VanityLevel': 4}, {'MSISDN': '0585699994', 'VanityLevel': 4}, {'MSISDN': '0585555922', 'VanityLevel': 4}, {'MSISDN': '0585122220', 'VanityLevel': 4}, {'MSISDN': '0585555329', 'VanityLevel': 4}, {'MSISDN': '0585022226', 'VanityLevel': 4}, {'MSISDN': '0585666634', 'VanityLevel': 4}, {'MSISDN': '0585555

KeyboardInterrupt: 

In [46]:
import requests
import pandas as pd
import time
import random
from datetime import datetime

# Virgin Mobile number search API
URL = "https://wsc.virginmobile.ae/numbers/search"

HEADERS = {
    "Accept": "application/json",
    "Channel-Id": "Web",
    "ChannelID": "Web",
    "Content-Type": "application/json",
    "Origin": "https://www.virginmobile.ae",
    "Referer": "https://www.virginmobile.ae/",
    "Session-Id": "6112a785-ab05-488f-baca-01983e3cc2cf",  # update if needed
    "User-Agent": "Mozilla/5.0",
    "Virgin-Device-Id": "730899726",
    "Virgin-Device-Os": "Web",
    "Virgin-Lang": "en",
    "accept-version": "v34",
}

VANITY_LEVEL = 4
PER_PAGE = 1000  # API seems to allow up to 1000
EXCEL_FILE = f"vanity_level_{VANITY_LEVEL}_numbers.xlsx"

# ---- Uniqueness trackers ----
unique_msisdns = set()          # the set you asked for (ALL unique numbers go here)
records_by_msisdn = {}          # keeps the first full record seen for each unique number

page = 1
start_time = datetime.now()
consecutive_no_new_pages = 0    # stop if pages stop yielding anything new
MAX_NO_NEW_PAGES = 2            # tweak if needed

def extract_msisdn(item):
    """Be tolerant to key naming — prefer 'MSISDN' but fall back if API changes."""
    return item.get("MSISDN") or item.get("msisdn") or item.get("number") or item.get("msisdn_number")

while True:
    try:
        payload = {
            "per_page": PER_PAGE,
            "page": page,
            "pattern": "",  # empty = no filter; change if you want narrower searches
            "vanity_level_ids": "1,2,3,4,10",
        }

        resp = requests.post(URL, headers=HEADERS, json=payload, timeout=30)
        resp.raise_for_status()
        body = resp.json()
        data = body.get("numbers", []) or body.get("data", [])  # be defensive

        if not data:
            print(f"No more data on page {page}. Ending fetch.")
            break

        before = len(unique_msisdns)
        for item in data:
            msisdn = extract_msisdn(item)
            if not msisdn:
                continue
            if msisdn not in unique_msisdns:
                unique_msisdns.add(msisdn)
                # keep one representative full record per unique number
                records_by_msisdn[msisdn] = {
                    "MSISDN": msisdn,
                    "vanity_level": VANITY_LEVEL,
                    "raw_data": item,
                }

        added = len(unique_msisdns) - before
        print(f"[Page {page}] Received {len(data)} rows, +{added} new uniques "
              f"(total uniques: {len(unique_msisdns)})")

        if added == 0:
            consecutive_no_new_pages += 1
            if consecutive_no_new_pages >= MAX_NO_NEW_PAGES:
                print(f"No new uniques for {MAX_NO_NEW_PAGES} consecutive pages. Stopping.")
                break
        else:
            consecutive_no_new_pages = 0

        page += 1
        # polite pacing to avoid hammering the API
        # time.sleep(random.uniform(0.2, 0.5))

    except requests.RequestException as e:
        print(f"HTTP error on page {page}: {e}. Retrying after a short pause...")
        # time.sleep(1.5)
    except Exception as e:
        print(f"Unexpected error on page {page}: {e}. Continuing...")
        # time.sleep(0.5)

# ---- Build final DataFrame from the unique set ----
df = pd.DataFrame(list(records_by_msisdn.values()))

# Optional: sort for readability
if not df.empty:
    df = df.sort_values(by="MSISDN").reset_index(drop=True)

# Save to Excel
df.to_excel(EXCEL_FILE, index=False)

print("\n--- Fetch Complete ---")
print(f"Total UNIQUE numbers: {len(unique_msisdns)}")
print(f"Saved to Excel: {EXCEL_FILE}")
print(f"🕒 Total Time: {datetime.now() - start_time}")


[Page 1] Received 1000 rows, +1000 new uniques (total uniques: 1000)
[Page 2] Received 1000 rows, +988 new uniques (total uniques: 1988)
[Page 3] Received 1000 rows, +971 new uniques (total uniques: 2959)
[Page 4] Received 1000 rows, +961 new uniques (total uniques: 3920)
[Page 5] Received 1000 rows, +939 new uniques (total uniques: 4859)
[Page 6] Received 1000 rows, +935 new uniques (total uniques: 5794)
[Page 7] Received 1000 rows, +937 new uniques (total uniques: 6731)
[Page 8] Received 1000 rows, +921 new uniques (total uniques: 7652)
[Page 9] Received 1000 rows, +902 new uniques (total uniques: 8554)
[Page 10] Received 1000 rows, +881 new uniques (total uniques: 9435)
[Page 11] Received 1000 rows, +885 new uniques (total uniques: 10320)
[Page 12] Received 1000 rows, +884 new uniques (total uniques: 11204)
[Page 13] Received 1000 rows, +854 new uniques (total uniques: 12058)
[Page 14] Received 1000 rows, +838 new uniques (total uniques: 12896)
[Page 15] Received 1000 rows, +822 new

In [48]:
pwd

'/Users/apple/Desktop/mcp'

In [54]:
# Vanity number pattern miner for "vanity_level_4_numbers.xlsx"
# - Loads the Excel
# - Normalizes each entry to digits
# - Keeps numbers that start with '585' (as requested)
# - Extracts the "tail" after 585
# - Detects memorable/regex-friendly patterns in the tail
# - Scores by memorability and exports results
#
# If the file isn't found, this will explain what to do.

import os
import re
import math
import pandas as pd
from pathlib import Path
from typing import List, Dict, Tuple
from collections import Counter
import numpy as np
from caas_jupyter_tools import display_dataframe_to_user

# ----------------------
# Config
# ----------------------
INPUT_CANDIDATES = [
  "/Users/apple/Desktop/mcp/vanity_level_4_numbers.xlsx"
]
OUTPUT_XLSX = "/mnt/data/vanity_candidates_scored.xlsx"
OUTPUT_CSV  = "/mnt/data/vanity_candidates_scored.csv"


def load_first_existing_excel(paths: List[str]) -> pd.DataFrame:
    for p in paths:
        if os.path.exists(p):
            try:
                df = pd.read_excel(p)
                df["__source_path"] = p
                return df
            except Exception as e:
                # try openpyxl engine fallback
                try:
                    df = pd.read_excel(p, engine="openpyxl")
                    df["__source_path"] = p
                    return df
                except Exception as e2:
                    continue
    # If none loaded, return empty with hint columns
    return pd.DataFrame()


def extract_digits(s: str) -> str:
    if pd.isna(s):
        return ""
    return re.sub(r"\D+", "", str(s))


def normalize_number(raw: str) -> str:
    """
    Normalize to something that starts with 585 if present.
    Strategy:
      - strip all non-digits
      - if it contains '585' and starts with it, keep as-is
      - else if it contains '585' somewhere (e.g., country code prefix), try to trim prefix until '585'
    """
    digits = extract_digits(raw)
    if digits.startswith("585"):
        return digits
    # If has '585' later, cut leading stuff
    idx = digits.find("585")
    if idx != -1:
        return digits[idx:]
    return digits  # fallback


def classify_tail(tail: str) -> Tuple[List[str], Dict[str, str]]:
    """
    Return (labels, details) for the tail string after '585'.
    We support variable tail lengths, but many patterns assume len >= 4.
    """
    labels = []
    details = {}

    # Basic chunks
    n = tail
    L = len(n)

    # Helper lambdas
    is_ascending = lambda s: all(int(s[i+1]) - int(s[i]) == 1 for i in range(len(s)-1))
    is_descending = lambda s: all(int(s[i]) - int(s[i+1]) == 1 for i in range(len(s)-1))
    all_same = lambda s: len(set(s)) == 1

    # --- Single regex classics ---
    # 6 of same digit (e.g., 000000)
    if re.fullmatch(r"(\d)\1{5}", n or ""):
        labels.append("xxxxxx-same-digit")  # e.g., 777777

    # (ab)(ab)(ab) where ab are any two digits (2-2-2 repeat)
    if re.fullmatch(r"(\d{2})\1\1", n or ""):
        labels.append("ab-ab-ab (2x2x2 repeat)")

    # (abc)(abc) (3-3 repeat)
    if re.fullmatch(r"(\d{3})\1", n or ""):
        labels.append("abc-abc (3x3 repeat)")

    # (aaa)(bbb) two triples of same digit (e.g., 000111)
    if re.fullmatch(r"(\d)\1{2}(\d)\2{2}", n or ""):
        labels.append("aaa-bbb (triples)")

    # Palindrome (reads same forward/backward), length >= 4
    if L >= 4 and n == n[::-1]:
        labels.append("palindrome")

    # Sandwich patterns like ABAABA (e.g., 121121, 343343). A=any digit, B=any digit
    if re.fullmatch(r"(\d)(\d)\1\1\2\1", n or ""):
        labels.append("ABA-ABA")

    # AABBCC (pairs of digits in order)
    if re.fullmatch(r"(\d)\1(\d)\2(\d)\3", n or ""):
        labels.append("AA-BB-CC")

    # ABCCBA (mirror pairs)
    if re.fullmatch(r"(\d)(\d)(\d)\3\2\1", n or ""):
        labels.append("ABC-CBA (mirror)")

    # Ascending/descending runs length >= 5
    if L >= 5 and is_ascending(n):
        labels.append("ascending-run")
    if L >= 5 and is_descending(n):
        labels.append("descending-run")

    # Repeated digit with one break like 000100 (highly memorable)
    if re.fullmatch(r"(\d)\1{2}\d\1{2}", n or ""):
        labels.append("XXXYXX style")

    # ---- Block patterns assuming 9-digit final number (585 + 6 tail) ----
    # Interpret as 3-3 blocks or 2-2-2 when L==6
    if L == 6:
        b1, b2 = n[:3], n[3:]
        if int(b2) - int(b1) == 1:
            labels.append("3-3 consecutive blocks (n,n+1)")
        if int(b2) - int(b1) == 100:
            labels.append("3-3 +100 staircase")
        # 2-2-2 consecutive blocks like 12 13 14 or 65 75 85 (step on tens)
        c1, c2, c3 = n[:2], n[2:4], n[4:]
        try:
            if int(c2) - int(c1) == 1 and int(c3) - int(c2) == 1:
                labels.append("2-2-2 consecutive (n,n+1,n+2)")
        except ValueError:
            pass

    # ---- Density-based patterns: many repeats or few unique digits ----
    unique_count = len(set(n))
    if L >= 4 and unique_count <= max(1, L // 3):
        labels.append("low-entropy (few unique digits)")

    # ---- Ending/starting patterns (000, 1111 etc.) ----
    if re.search(r"(000|111|222|333|444|555|666|777|888|999)$", n or ""):
        labels.append("ends-with-triple")
    if re.match(r"^(000|111|222|333|444|555|666|777|888|999)", n or ""):
        labels.append("starts-with-triple")

    # --- Details for blocks when L==6 ---
    if L == 6:
        details["blocks_3_3"] = f"{n[:3]} {n[3:]}"
        details["blocks_2_2_2"] = f"{n[:2]} {n[2:4]} {n[4:]}"
    return labels, details


# Memorability scoring (tweakable weights)
WEIGHTS = {
    "abc-abc (3x3 repeat)": 9,
    "ab-ab-ab (2x2x2 repeat)": 9,
    "xxxxxx-same-digit": 9,
    "aaa-bbb (triples)": 7,
    "ABC-CBA (mirror)": 7,
    "palindrome": 6,
    "ascending-run": 6,
    "descending-run": 6,
    "3-3 consecutive blocks (n,n+1)": 6,
    "3-3 +100 staircase": 5,
    "2-2-2 consecutive (n,n+1,n+2)": 5,
    "AA-BB-CC": 5,
    "ABA-ABA": 5,
    "XXXYXX style": 4,
    "low-entropy (few unique digits)": 3,
    "starts-with-triple": 3,
    "ends-with-triple": 3,
}


def score_labels(labels: List[str]) -> int:
    return sum(WEIGHTS.get(lbl, 0) for lbl in labels)


# ------------ Run -------------
df_raw = load_first_existing_excel(INPUT_CANDIDATES)

if df_raw.empty:
    msg = (
        "❌ I couldn't find 'vanity_level_4_numbers.xlsx'. \n\n"
        "Please upload the Excel file to this chat or place it at /mnt/data/vanity_level_4_numbers.xlsx, "
        "then ask me to re-run. The file should contain a column with the numbers."
    )
    print(msg)
else:
    # Pick a best-guess column: prefer ones named like 'number', 'msisdn', 'mobile', else first column
    candidate_cols = [c for c in df_raw.columns if re.search(r"(number|msisdn|mobile|phone)", str(c), re.I)]
    use_col = candidate_cols[0] if candidate_cols else df_raw.columns[0]

    work = df_raw.copy()
    work["raw_number"] = work[use_col].astype(str)
    work["normalized"] = work["raw_number"].map(normalize_number)
    work["starts_585"] = work["normalized"].str.startswith("585")
    work = work[work["starts_585"]].copy()

    # tail after 585
    work["tail"] = work["normalized"].str.replace(r"^585", "", regex=True)

    # Only keep tails with at least 4 digits (tune if needed). Most UAE patterns target total length 9 => tail len 6.
    work = work[work["tail"].str.len() >= 4].copy()

    # Classify
    labels_list = []
    details_blocks_33 = []
    details_blocks_222 = []

    for t in work["tail"]:
        labels, details = classify_tail(t)
        labels_list.append(labels)
        details_blocks_33.append(details.get("blocks_3_3", ""))
        details_blocks_222.append(details.get("blocks_2_2_2", ""))

    work["patterns"] = labels_list
    work["pattern_count"] = work["patterns"].apply(len)
    work["score"] = work["patterns"].apply(score_labels)
    work["blocks_3_3"] = details_blocks_33
    work["blocks_2_2_2"] = details_blocks_222

    # Useful string view
    work["patterns_str"] = work["patterns"].apply(lambda xs: ", ".join(xs))

    # Sort by score desc, then by fewer unique digits, then by tail lexicographically
    work["uniq_digits"] = work["tail"].apply(lambda s: len(set(s)))
    work = work.sort_values(["score", "pattern_count", "uniq_digits", "tail"], ascending=[False, False, True, True])

    # Save
    work_out = work[
        [
            "raw_number",
            "normalized",
            "tail",
            "patterns_str",
            "score",
            "pattern_count",
            "blocks_3_3",
            "blocks_2_2_2",
            "__source_path",
        ]
    ].reset_index(drop=True)

    # Export
    work_out.to_excel(OUTPUT_XLSX, index=False)
    work_out.to_csv(OUTPUT_CSV, index=False)

    # Show a small preview to the user and also the pattern histogram
    display_dataframe_to_user("Top Vanity Candidates (scored)", work_out.head(100))

    # Pattern counts
    all_labels = [lbl for labels in labels_list for lbl in labels]
    pattern_counts = pd.Series(Counter(all_labels)).sort_values(ascending=False).reset_index()
    pattern_counts.columns = ["pattern", "count"]
    display_dataframe_to_user("Pattern Frequency", pattern_counts)

    print("✅ Done. Exports created:\n"
          f"- [XLSX] {OUTPUT_XLSX}\n"
          f"- [CSV]  {OUTPUT_CSV}\n")


ModuleNotFoundError: No module named 'caas_jupyter_tools'

In [52]:
!pip install caas_jupyter_tools

ERROR: Could not find a version that satisfies the requirement caas_jupyter_tools (from versions: none)
ERROR: No matching distribution found for caas_jupyter_tools


In [58]:
import os
import re
import pandas as pd
from collections import Counter

# ----------------------
# Config
# ----------------------
INPUT_FILE = "/Users/apple/Desktop/mcp/vanity_level_4_numbers.xlsx"
OUTPUT_XLSX = "/Users/apple/Desktop/mcp/vanity_candidates_scored.xlsx"
OUTPUT_CSV  = "/Users/apple/Desktop/mcp/vanity_candidates_scored.csv"


def extract_digits(s: str) -> str:
    if pd.isna(s):
        return ""
    return re.sub(r"\D+", "", str(s))


def normalize_number(raw: str) -> str:
    digits = extract_digits(raw)
    if digits.startswith("585"):
        return digits
    idx = digits.find("585")
    if idx != -1:
        return digits[idx:]
    return digits


def classify_tail(tail: str):
    labels = []

    n = tail
    L = len(n)

    if re.fullmatch(r"(\d)\1{5}", n):
        labels.append("xxxxxx-same-digit")
    if re.fullmatch(r"(\d{2})\1\1", n):
        labels.append("ab-ab-ab")
    if re.fullmatch(r"(\d{3})\1", n):
        labels.append("abc-abc")
    if re.fullmatch(r"(\d)\1{2}(\d)\2{2}", n):
        labels.append("aaa-bbb")
    if L >= 4 and n == n[::-1]:
        labels.append("palindrome")
    if re.fullmatch(r"(\d)(\d)(\d)\3\2\1", n):
        labels.append("ABC-CBA mirror")

    return labels


WEIGHTS = {
    "xxxxxx-same-digit": 9,
    "ab-ab-ab": 9,
    "abc-abc": 9,
    "aaa-bbb": 7,
   
}


def score_labels(labels):
    return sum(WEIGHTS.get(lbl, 0) for lbl in labels)


# ------------ Run -------------
df_raw = pd.read_excel(INPUT_FILE)

# Pick a column with numbers
candidate_cols = [c for c in df_raw.columns if re.search(r"(number|msisdn|mobile|phone)", str(c), re.I)]
use_col = candidate_cols[0] if candidate_cols else df_raw.columns[0]

work = df_raw.copy()
work["raw_number"] = work[use_col].astype(str)
work["normalized"] = work["raw_number"].map(normalize_number)
work = work[work["normalized"].str.startswith("585")].copy()
work["tail"] = work["normalized"].str.replace(r"^585", "", regex=True)

labels_list = []
for t in work["tail"]:
    labels = classify_tail(t)
    labels_list.append(labels)

work["patterns"] = labels_list
work["score"] = work["patterns"].apply(score_labels)
work["patterns_str"] = work["patterns"].apply(lambda xs: ", ".join(xs))

work = work.sort_values(["score", "patterns_str"], ascending=[False, True])

# Save
work.to_excel(OUTPUT_XLSX, index=False)
work.to_csv(OUTPUT_CSV, index=False)

print("✅ Done.")
print("Saved to:")
print(f" - {OUTPUT_XLSX}")
print(f" - {OUTPUT_CSV}")

print("\nTop 20 vanity numbers:")
print(work[["raw_number", "tail", "patterns_str", "score"]].head(20))


✅ Done.
Saved to:
 - /Users/apple/Desktop/mcp/vanity_candidates_scored.xlsx
 - /Users/apple/Desktop/mcp/vanity_candidates_scored.csv

Top 20 vanity numbers:
     raw_number    tail                patterns_str  score
4694  585464646  464646                    ab-ab-ab      9
9644  585583583  583583                     abc-abc      9
329   585133331  133331  palindrome, ABC-CBA mirror      6
365   585166661  166661  palindrome, ABC-CBA mirror      6
389   585211112  211112  palindrome, ABC-CBA mirror      6
517   585233332  233332  palindrome, ABC-CBA mirror      6
530   585244442  244442  palindrome, ABC-CBA mirror      6
580   585266662  266662  palindrome, ABC-CBA mirror      6
626   585322223  322223  palindrome, ABC-CBA mirror      6
755   585344443  344443  palindrome, ABC-CBA mirror      6
807   585366663  366663  palindrome, ABC-CBA mirror      6
842   585399993  399993  palindrome, ABC-CBA mirror      6
853   585411114  411114  palindrome, ABC-CBA mirror      6
871   585422224  

In [60]:
import os
import re
import pandas as pd
from collections import Counter

# ----------------------
# Config
# ----------------------
INPUT_FILE = "/Users/apple/Desktop/mcp/vanity_level_4_numbers.xlsx"
OUTPUT_XLSX = "/Users/apple/Desktop/mcp/vanity_candidates_scored.xlsx"
OUTPUT_CSV  = "/Users/apple/Desktop/mcp/vanity_candidates_scored.csv"
OUTPUT_TOP200_CSV = "/Users/apple/Desktop/mcp/vanity_top200.csv"

# If you want to only consider 9-digit numbers total (585 + 6 tail), set this True
TAIL_LEN_6_ONLY = False  # set True if UAE 9-digit normalization is required


def extract_digits(s: str) -> str:
    if pd.isna(s):
        return ""
    return re.sub(r"\D+", "", str(s))


def normalize_number(raw: str) -> str:
    """
    Normalize to start from the first occurrence of '585' (your fixed prefix).
    """
    digits = extract_digits(raw)
    if digits.startswith("585"):
        return digits
    idx = digits.find("585")
    if idx != -1:
        return digits[idx:]
    return digits


def is_ascending(s: str) -> bool:
    if len(s) < 2: return False
    try:
        return all(int(s[i+1]) - int(s[i]) == 1 for i in range(len(s)-1))
    except ValueError:
        return False


def is_descending(s: str) -> bool:
    if len(s) < 2: return False
    try:
        return all(int(s[i]) - int(s[i+1]) == 1 for i in range(len(s)-1))
    except ValueError:
        return False


def classify_tail(tail: str):
    """
    Return (labels, details) for the tail after '585'.
    Covers many memorable forms:
      - xxxxxx same digit
      - ab-ab-ab (2x2x2 repeat)
      - abc-abc (3x3 repeat)
      - aaa-bbb (triples)
      - AA-BB-CC
      - ABC-CBA (mirror)
      - ABA-ABA
      - palindrome
      - ascending/descending run
      - 3-3 consecutive blocks (n,n+1)
      - 3-3 +100 staircase
      - 2-2-2 consecutive (n,n+1,n+2)
      - XXXYXX style
      - starts/ends with triple
      - low-entropy (few unique digits)
    """
    labels = []
    details = {}

    n = tail
    L = len(n)
    if L == 0:
        return labels, details

    # ------- Core regex families -------
    if re.fullmatch(r"(\d)\1{5}", n):
        labels.append("xxxxxx-same-digit")

    if re.fullmatch(r"(\d{2})\1\1", n):
        labels.append("ab-ab-ab (2x2x2 repeat)")

    if re.fullmatch(r"(\d{3})\1", n):
        labels.append("abc-abc (3x3 repeat)")

    if re.fullmatch(r"(\d)\1{2}(\d)\2{2}", n):
        labels.append("aaa-bbb (triples)")

    if L >= 4 and n == n[::-1]:
        labels.append("palindrome")

    if re.fullmatch(r"(\d)(\d)(\d)\3\2\1", n):
        labels.append("ABC-CBA (mirror)")

    if re.fullmatch(r"(\d)\1(\d)\2(\d)\3", n):
        labels.append("AA-BB-CC")

    if re.fullmatch(r"(\d)(\d)\1\1\2\1", n):
        labels.append("ABA-ABA")

    # Asc/Desc runs (length >=5 is usually notable)
    if L >= 5 and is_ascending(n):
        labels.append("ascending-run")
    if L >= 5 and is_descending(n):
        labels.append("descending-run")

    # XXXYXX style (two groups of three with a single break in the middle: e.g., 000100)
    if re.fullmatch(r"(\d)\1{2}\d\1{2}", n):
        labels.append("XXXYXX style")

    # Starts/ends with triple
    if re.search(r"(000|111|222|333|444|555|666|777|888|999)$", n):
        labels.append("ends-with-triple")
    if re.match(r"^(000|111|222|333|444|555|666|777|888|999)", n):
        labels.append("starts-with-triple")

    # Low entropy (few unique digits)
    unique_count = len(set(n))
    if L >= 4 and unique_count <= max(1, L // 3):
        labels.append("low-entropy (few unique digits)")

    # Block interpretations when length==6 (most common UAE tail length)
    if L == 6:
        b1, b2 = n[:3], n[3:]
        details["blocks_3_3"] = f"{b1} {b2}"
        try:
            if int(b2) - int(b1) == 1:
                labels.append("3-3 consecutive blocks (n,n+1)")
            if int(b2) - int(b1) == 100:
                labels.append("3-3 +100 staircase")
        except ValueError:
            pass

        c1, c2, c3 = n[:2], n[2:4], n[4:]
        details["blocks_2_2_2"] = f"{c1} {c2} {c3}"
        try:
            if int(c2) - int(c1) == 1 and int(c3) - int(c2) == 1:
                labels.append("2-2-2 consecutive (n,n+1,n+2)")
        except ValueError:
            pass

    return labels, details


WEIGHTS = {
    "abc-abc (3x3 repeat)": 9,
    "ab-ab-ab (2x2x2 repeat)": 9,
    "xxxxxx-same-digit": 9,
    "aaa-bbb (triples)": 7,
    "ABC-CBA (mirror)": 7,
    "palindrome": 6,
    "ascending-run": 6,
    "descending-run": 6,
    "3-3 consecutive blocks (n,n+1)": 6,
    "3-3 +100 staircase": 5,
    "2-2-2 consecutive (n,n+1,n+2)": 5,
    "AA-BB-CC": 5,
    "ABA-ABA": 5,
    "XXXYXX style": 4,
    "low-entropy (few unique digits)": 3,
    "starts-with-triple": 3,
    "ends-with-triple": 3,
}


def score_labels(labels):
    return sum(WEIGHTS.get(lbl, 0) for lbl in labels)


# ------------ Run -------------
df_raw = pd.read_excel(INPUT_FILE)

# Pick a column with numbers
candidate_cols = [c for c in df_raw.columns if re.search(r"(number|msisdn|mobile|phone)", str(c), re.I)]
use_col = candidate_cols[0] if candidate_cols else df_raw.columns[0]

work = df_raw.copy()
work["raw_number"] = work[use_col].astype(str)
work["normalized"] = work["raw_number"].map(normalize_number)

# Keep those that start with 585
work = work[work["normalized"].str.startswith("585")].copy()

# Tail after 585
work["tail"] = work["normalized"].str.replace(r"^585", "", regex=True)

# Optional: enforce tails of length 6 (so total is 9 digits)
if TAIL_LEN_6_ONLY:
    work = work[work["tail"].str.len() == 6].copy()
else:
    # Still ignore very short tails (<4) as non-memorable
    work = work[work["tail"].str.len() >= 4].copy()

# Classify
labels_list = []
blocks33 = []
blocks222 = []
for t in work["tail"]:
    labels, details = classify_tail(t)
    labels_list.append(labels)
    blocks33.append(details.get("blocks_3_3", ""))
    blocks222.append(details.get("blocks_2_2_2", ""))

work["patterns"] = labels_list
work["pattern_count"] = work["patterns"].apply(len)
work["score"] = work["patterns"].apply(score_labels)
work["patterns_str"] = work["patterns"].apply(lambda xs: ", ".join(xs))
work["blocks_3_3"] = blocks33
work["blocks_2_2_2"] = blocks222

# Extra tie-breakers: fewer unique digits (tend to be catchier), then tail lexicographically
work["uniq_digits"] = work["tail"].apply(lambda s: len(set(s)))

# Sort: score desc, pattern_count desc, uniq_digits asc, tail asc
work = work.sort_values(
    ["score", "pattern_count", "uniq_digits", "tail"],
    ascending=[False, False, True, True]
).reset_index(drop=True)

# Save full ranked list
work.to_excel(OUTPUT_XLSX, index=False)
work.to_csv(OUTPUT_CSV, index=False)

# Top 200
top200 = work.head(200)
top200.to_csv(OUTPUT_TOP200_CSV, index=False)

print("✅ Done.")
print("Saved to:")
print(f" - {OUTPUT_XLSX}")
print(f" - {OUTPUT_CSV}")
print(f" - {OUTPUT_TOP200_CSV} (Top 200)")

print("\nTop 200 preview:")
print(top200[["raw_number", "normalized", "tail", "patterns_str", "score"]].head(20).to_string(index=False))


✅ Done.
Saved to:
 - /Users/apple/Desktop/mcp/vanity_candidates_scored.xlsx
 - /Users/apple/Desktop/mcp/vanity_candidates_scored.csv
 - /Users/apple/Desktop/mcp/vanity_top200.csv (Top 200)

Top 200 preview:
raw_number normalized   tail                                                  patterns_str  score
 585133331  585133331 133331 palindrome, ABC-CBA (mirror), low-entropy (few unique digits)     16
 585166661  585166661 166661 palindrome, ABC-CBA (mirror), low-entropy (few unique digits)     16
 585211112  585211112 211112 palindrome, ABC-CBA (mirror), low-entropy (few unique digits)     16
 585233332  585233332 233332 palindrome, ABC-CBA (mirror), low-entropy (few unique digits)     16
 585244442  585244442 244442 palindrome, ABC-CBA (mirror), low-entropy (few unique digits)     16
 585266662  585266662 266662 palindrome, ABC-CBA (mirror), low-entropy (few unique digits)     16
 585322223  585322223 322223 palindrome, ABC-CBA (mirror), low-entropy (few unique digits)     16
 58534444